In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import torch
import re
from sklearn.metrics import balanced_accuracy_score, roc_auc_score,accuracy_score
from Constants import *
from Preprocessing import *
from Models import *
from Utils import *
pd.set_option('display.max_rows', 200)



In [3]:
pd.read_csv('../data/digital_twin_ln_monograms.csv')

,id,L1A,L1B,L2A,L2B,L3,L4,L5A,L5B,L6,...,R1A,R1B,R2A,R2B,R3,R4,R5A,R5B,R6,RRPLN
0,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,10201,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
532,10202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
533,10203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
534,10204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
data = DTDataset(use_smote=True)
data.processed_df.T
# data.processed_df#.shape, len(data.processed_df.index.unique())

/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/usr/local/lib/python3.8/dist-packages/imblearn/over_sampling/_smote/base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


id,3,5,6,7,8,9,10,11,13,14,...,10196,10197,10198,10199,10200,10201,10202,10203,10204,10205
hpv,1,0,1,1,1,1,-1,1,0,1,...,0,1,-1,0,1,1,0,1,0,1
age,55.969444,20.95,69.930556,72.319444,59.730556,60.083333,67.708333,57.858333,51.758333,56.25,...,47.619444,50.163889,70.888889,67.825,56.336111,49.566667,48.705556,77.116667,45.95,49.733333
packs_per_year,0.0,38.0,35.0,0.0,0.0,0.0,40.0,44.0,0.0,40.0,...,5.0,0.0,50.0,0.0,0.0,30.0,30.0,0.0,5.0,0.0
smoking_status,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.5,0.0,0.5,0.0,0.0,1.0,1.0,0.0,0.5,0.0
gender,1,1,0,1,1,1,1,1,1,1,...,0,1,0,1,1,1,1,1,1,1
Aspiration rate Pre-therapy,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
total_dose,66.0,72.0,70.0,70.0,66.0,66.0,69.96,70.0,70.0,70.0,...,70.0,72.0,66.0,70.0,69.96,70.0,72.0,70.0,69.96,69.96
dose_fraction,2.2,1.8,2.121212,2.121212,2.2,2.2,2.12,2.121212,2.0,2.121212,...,2.121212,1.8,2.2,2.121212,2.12,2.121212,1.714286,2.333333,2.12,2.12
bilateral,False,False,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
White/Caucasion,True,True,True,True,False,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True


In [5]:
get_tt_split()[0]

/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


[5,
 6,
 8,
 11,
 13,
 14,
 15,
 16,
 17,
 18,
 21,
 23,
 24,
 26,
 27,
 28,
 32,
 33,
 37,
 38,
 39,
 40,
 41,
 42,
 48,
 49,
 50,
 51,
 53,
 55,
 56,
 57,
 60,
 64,
 65,
 67,
 69,
 71,
 74,
 75,
 78,
 79,
 80,
 81,
 82,
 87,
 88,
 91,
 94,
 96,
 99,
 103,
 109,
 116,
 119,
 120,
 121,
 125,
 148,
 150,
 153,
 178,
 181,
 183,
 185,
 186,
 188,
 191,
 192,
 193,
 196,
 197,
 198,
 200,
 201,
 203,
 204,
 205,
 206,
 207,
 210,
 212,
 213,
 214,
 216,
 218,
 219,
 220,
 221,
 222,
 223,
 225,
 226,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 237,
 238,
 239,
 240,
 241,
 243,
 244,
 246,
 247,
 248,
 249,
 251,
 252,
 253,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 265,
 266,
 269,
 270,
 273,
 275,
 276,
 277,
 278,
 280,
 281,
 282,
 283,
 285,
 289,
 2000,
 2002,
 2003,
 2004,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2016,
 2018,
 2021,
 2022,
 2023,
 2025,
 2027,
 2028,
 2030,
 2033,
 5000,
 5002,
 5004,
 5005,
 5006,
 5008,
 5009,
 5010,
 5011,
 5012,
 5

In [6]:
get_tt_split(resample_training=True)[0]

/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


array([  266,   246, 10044,  5055,  5058,  5037,   213,    23, 10175,
       10004,   183, 10129, 10115,   261,    81, 10116, 10033,    28,
         244,   220,    53, 10070, 10019,    51,     6, 10035,   260,
       10004,    60, 10011,   246,    91,   220, 10057,    67, 10162,
         252, 10074, 10198, 10175, 10184,  2028,  5028,  5091,    49,
       10004, 10052,   276,   259,    13,   251,  5030,  5070,   200,
         263,  5079,  5034, 10115, 10034,  2008,  2027, 10183, 10099,
       10002,    78,   116,  5010, 10038,   253,   238,  5073, 10024,
       10113, 10027, 10080, 10071,  5070,    51, 10055,    48,  5113,
         231, 10168,  5060, 10119,    15,    48,   181, 10080, 10044,
       10009,  5110, 10048,   214,  5088,    99,  5012,    87, 10186,
         263,  5037, 10073,   269,   253,    56, 10107,  5025,   196,
           6, 10159,   206, 10009, 10022, 10029, 10113, 10047,   109,
        2004,  5042,   183,  5004,   225, 10189,  2000, 10201, 10148,
          32,   205,

In [7]:
def nllloss(ytrue,ypred):
    #nll loss with argmax added in
    loss = torch.nn.NLLLoss()
    return loss(ypred,ytrue.argmax(axis=1))

def state_loss(ytrue,ypred,weights=[1,1,1,1]):
    pd_loss = nllloss(ytrue[0],ypred[0])*weights[0]
    nd_loss = nllloss(ytrue[1],ypred[1])*weights[1]
    mod_loss = nllloss(ytrue[2],ypred[2])*weights[2]
    loss = pd_loss + nd_loss + mod_loss
    dlt_true = ytrue[3]
    dlt_pred = ypred[3]
    ndlt = dlt_true.shape[1]
#     nloss = torch.nn.NLLLoss()
    bce = torch.nn.BCELoss()
    for i in range(ndlt):
        dlt_loss = bce(dlt_pred[:,i].view(-1),dlt_true[:,i].view(-1))
        loss += dlt_loss*weights[3]/ndlt
    return loss

def outcome_loss(ytrue,ypred,weights=[1,1,1]):
    loss = 0
    nloss = torch.nn.BCELoss()
    for i in range(len(weights)):
        iloss = nloss(ypred[:,i],ytrue[i])*weights[i]
        loss += iloss
    return loss

def mc_metrics(yt,yp,numpy=False,is_dlt=False):
    if not numpy:
        yt = yt .cpu().detach().numpy()
        yp = yp.cpu().detach().numpy()
    #dlt prediction (binary)
    if is_dlt:
        acc = accuracy_score(yt,yp>.5)
        if yt.sum() > 1:
            auc = roc_auc_score(yt,yp)
        else:
            auc=-1
        error = np.mean((yt-yp)**2)
        return {'accuracy': acc, 'mse': error, 'auc': auc}
    #this is a catch for when I se the dlt prediction format (encoded integer ordinal, predict as a categorical and take the argmax)
    elif yt.ndim > 1:
        try:
            bacc = balanced_accuracy_score(yt.argmax(axis=1),yp.argmax(axis=1))
        except:
            bacc = -1
        try:
            roc_micro = roc_auc_score(yt,yp,average='micro')
        except:
            roc_micro=-1
        try:
            roc_macro = roc_auc_score(yt,yp,average='macro')
        except:
            roc_macro = -1
        return {'accuracy': bacc, 'roc_micro': roc_micro,'roc_macro': roc_macro}
    #outcomes (binary)
    else:
        if yp.ndim > 1:
            yp = yp.argmax(axis=1)
        try:
            bacc = accuracy_score(yt,yp)
        except:
            bacc = -1
        try:
            roc = roc_auc_score(yt,yp)
        except:
            roc = -1
        error = np.mean((yt-yp)**2)
        return {'accuracy': bacc, 'mse': error, 'auc': roc}

def state_metrics(ytrue,ypred,numpy=False):
    pd_metrics = mc_metrics(ytrue[0],ypred[0],numpy=numpy)
    nd_metrics = mc_metrics(ytrue[1],ypred[1],numpy=numpy)
    mod_metrics = mc_metrics(ytrue[1],ypred[1],numpy=numpy)
    
    dlt_metrics = []
    dlt_true = ytrue[3]
    dlt_pred = ypred[3]
    ndlt = dlt_true.shape[1]
    nloss = torch.nn.NLLLoss()
    for i in range(ndlt):
        dm = mc_metrics(dlt_true[:,i],dlt_pred[:,i].view(-1),is_dlt=True)
        dlt_metrics.append(dm)
    dlt_acc =[d['accuracy'] for d in dlt_metrics]
    dlt_error = [d['mse'] for d in dlt_metrics]
    dlt_auc = [d['auc'] for d in dlt_metrics]
    
    acc_mean = np.mean([a for a in dlt_acc if a >= 0])
    auc_mean = np.mean([a for a in dlt_auc if a >= 0])
    results = {'pd': pd_metrics,'nd': nd_metrics,'mod': mod_metrics,
               'dlts': {'accuracy': dlt_acc,'accuracy_mean': acc_mean,'auc': dlt_auc,'auc_mean': auc_mean}
              }
    return results

def outcome_metrics(ytrue,ypred,numpy=False):
    res = {}
    for i, outcome in enumerate(Const.outcomes):
        metrics = mc_metrics(ytrue[i],ypred[:,i])
        res[outcome] = metrics
    return res

def transition_sample(state):
    ids = get_dt_ids()
    
    train_ids, test_ids = get_tt_split()
    
    dataset = DTDataset()
    
    #only train on people with  IC for state 1 since other people can't have any outcomes otherwise
    require = None
    if state == 1:
        require = Const.decisions[0] #we don't expect a state update if there is no treatment
        valid_ids = dataset.get_input_state(require=require).index.values
        train_ids = [t for t in train_ids if t in valid_ids]
        test_ids = [t for t in test_ids if t in valid_ids]
    xtrain = dataset.get_input_state(step=state,ids=train_ids,require=require)
    xtest = dataset.get_input_state(step=state,ids=test_ids,require=require)
    ytrain = dataset.get_intermediate_outcomes(step=state,ids=train_ids,require=require)
    ytest = dataset.get_intermediate_outcomes(step=state,ids=test_ids,require=require)

    xtrain = df_to_torch(xtrain)
    xtest = df_to_torch(xtest)
    ytrain = [df_to_torch(t) for t in ytrain]
    ytest= [df_to_torch(t) for t in ytest]
    return xtrain,xtest,ytrain,ytest

In [8]:
def train_state(model=None,
                model_args={},
                state=1,
                split=.7,
                lr=.0001,
                epochs=1000,
                patience=10,
                use_attention=False,
                weights=None,
                save_path='../data/models/',
                use_default_split=True,
                use_bagging_split=False,
                resample_training=False,#use bootstraping on training data after splitting
                n_validation_trainsteps=2,
                verbose=True,
                use_smote=True,
                file_suffix=''):
    
    ids = get_dt_ids()
    
    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)

    if use_smote:
        dataset = DTDataset(use_smote=True,smote_ids = train_ids)
        train_ids = [i for i in dataset.processed_df.index.values if i not in test_ids]
    else:
        dataset = DTDataset()
    
    #only train on people with  IC for state 1 since other people can't have any outcomes otherwise
    require = None
    if state == 1:
        require = Const.decisions[0] #we don't expect a state update if there is no treatment
        valid_ids = dataset.get_input_state(require=require).index.values
        train_ids = [t for t in train_ids if t in valid_ids]
        test_ids = [t for t in test_ids if t in valid_ids]
    xtrain = dataset.get_input_state(step=state,ids=train_ids,require=require)
    xtest = dataset.get_input_state(step=state,ids=test_ids,require=require)
    ytrain = dataset.get_intermediate_outcomes(step=state,ids=train_ids,require=require)
    ytest = dataset.get_intermediate_outcomes(step=state,ids=test_ids,require=require)
    

    if not use_attention:
        model_args = {k:v for k,v in model_args.items() if 'attention' not in k and 'embed_size' not in k}
    if state < 3:
        if model is None:
            if use_attention:
                model = OutcomeAttentionSimulator(xtrain.shape[1],state=state,**model_args)
            else:
                model = OutcomeSimulator(xtrain.shape[1],state=state,**model_args)
        lfunc = state_loss
        if weights is None:
            weights = [1,1,1,1]
    else:
        if model is None:
            if use_attention:
                model = EndpointAttentionSimulator(xtrain.shape[1],**model_args)
            else:
                model = EndpointSimulator(xtrain.shape[1],**model_args)
        if weights is None:
            weights = [1,1,1,1]
        lfunc = outcome_loss
        
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.set_device(device)
    
    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    save_file = save_path + 'model_' + model.identifier + '_split' + str(split) + '_resample' + str(resample_training) +  '_hash' + hashcode + file_suffix + '.tar'
    xtrain = df_to_torch(xtrain).to(device)
    xtest = df_to_torch(xtest).to(device)
    ytrain = [df_to_torch(t).to(device) for t in ytrain]
    ytest= [df_to_torch(t).to(device) for t in ytest]
    
    model.fit_normalizer(xtrain)
#     normalize = lambda x: (x - xtrain.mean(axis=0)+.01)/(xtrain.std(axis=0)+.01)
#     unnormalize = lambda x: (x * (xtrain.std(axis=0) +.01)) + xtrain.mean(axis=0) - .01
    
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    best_val_loss = 1000000000000000000000000000
    best_loss_metrics = {}
    last_epoch = False
    for epoch in range(epochs):
        
        model.train(True)
        optimizer.zero_grad()
        
        xtrain_sample = xtrain#[torch.randint(len(xtrain),(len(xtrain),) )]
        ypred = model(xtrain_sample)
        loss = lfunc(ytrain,ypred,weights=weights)

        loss.backward()
        optimizer.step()
        if verbose:
            print('epoch',epoch,'train loss',loss.item())
        
        model.eval()
        yval = model(xtest)
        val_loss = lfunc(ytest,yval,weights=weights)
        if state < 3:
            val_metrics = state_metrics(ytest,yval)
        else:
            val_metrics = outcome_metrics(ytest,yval)
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_loss_metrics = val_metrics
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if verbose:
            print('val loss',val_loss.item())
            print('______________')
        if steps_since_improvement > patience:
            break
    print('best loss',best_val_loss,best_loss_metrics)
    model.load_state_dict(torch.load(save_file))
    
    #train one step on validation data
    for i in range(n_validation_trainsteps):
        model.train()
        yval = model(xtest)
        val_loss = lfunc(ytest,yval,weights=weights)
        val_loss.backward()
        optimizer.step()
        torch.save(model.state_dict(),save_file)
    
    model.eval()
    return model,  best_val_loss, best_loss_metrics

In [9]:
def train_model_ensemble(model_arg_list,state,
                         n_bags=10,
                         smote_options=[True,False],
                         smote_baseline_options=[True,False],
                         weight_options=[None],
                         weight_baseline_options=[None],
                         **kwargs):
    resampled_models =[]
    base_models = []
    base_metrics = []
    resample_metrics = []
    base_losses = []
    resample_losses = []
    n_errors = 0
    for margs in model_arg_list:
        for s in smote_baseline_options:
            for w in weight_baseline_options:
                [base_model,blosses,bmetrics] = train_state(model_args=margs,state=state,
                                         resample_training=False,
                                         use_smote=smote_options,
                                         weights=w,
                                         verbose=False,**kwargs)
                base_models.append(base_model)
                base_metrics.append(bmetrics)
                base_losses.append(blosses)
        for n in range(n_bags):
            for s in smote_options:
                for w in weight_options:
                    #this can fail if I resample a bad distribution with no minority classes
                    done = False
                    while not done:
                        try:
                            [model,loss,metrics] = train_state(model_args=margs,
                                                state=state,resample_training=True,
                                                use_smote=s,
                                                weights=w,
                                                verbose=False,**kwargs)
                            resampled_models.append(model)
                            resample_metrics.append(metrics)
                            resample_losses.append(loss)
                            done=True
                            print('model_done',len(base_models)+len(resampled_models))
                        except Exception as e:
                            print('error training model')
                            print(e)
                            n_errors += 1
    print('done with',n_errors,'errors')
    return base_models,resampled_models

In [10]:
t1_arglist = [
     {'hidden_layers': [400,400], 'dropout': 0.9, 'input_dropout': 0.1},
    {'hidden_layers': [400], 'dropout': .7, 'input_dropout': .2}
]
m1_list, m1_ensemble = train_model_ensemble(t1_arglist,1,
                                            n_bags=5,
                                            weight_options=[[1,1,1,10],[2,2,0.1,1]],
                                            weight_baseline_options=[[1,1,1,10],[2,2,0.1,1]],
                                           )
from Models import *
emodel1 = TransitionEnsemble(m1_list,m1_ensemble)
test2 = emodel1(transition_sample(1)[0][0].view(1,-1).to(emodel1.get_device()))
test2

/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


here


/usr/local/lib/python3.8/dist-packages/imblearn/over_sampling/_smote/base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


best loss 5.822696685791016 {'pd': {'accuracy': 0.376984126984127, 'roc_micro': 0.7355371900826445, 'roc_macro': 0.5371700651586799}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.8662398921832885, 'roc_macro': 0.5190229392357052}, 'mod': {'accuracy': 0.3333333333333333, 'roc_micro': 0.8662398921832885, 'roc_macro': 0.5190229392357052}, 'dlts': {'accuracy': [0.7818181818181819, 0.9454545454545454, 0.9090909090909091, 1.0, 1.0, 0.8, 1.0, 0.8909090909090909], 'accuracy_mean': 0.915909090909091, 'auc': [0.5058139534883721, 0.37820512820512814, 0.56, -1, -1, 0.5537190082644627, -1, 0.41496598639455784], 'auc_mean': 0.4825408152705042}}


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


...

best loss 7.191603660583496 {'pd': {'accuracy': 0.4023529411764706, 'roc_micro': 0.7393543361606371, 'roc_macro': 0.5772977472359736}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.882277097902098, 'roc_macro': 0.5134968634090714}, 'mod': {'accuracy': 0.3333333333333333, 'roc_micro': 0.882277097902098, 'roc_macro': 0.5134968634090714}, 'dlts': {'accuracy': [0.8425925925925926, 0.9537037037037037, 0.8703703703703703, 1.0, 0.9907407407407407, 0.8518518518518519, 0.9814814814814815, 0.8148148148148148], 'accuracy_mean': 0.9131944444444444, 'auc': [0.43244990303813835, 0.6757281553398058, 0.4414893617021277, -1, -1, 0.4993206521739131, 0.6462264150943396, 0.6], 'auc_mean': 0.5492024145580542}}
model_done 47


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


best loss 3.693408727645874 {'pd': {'accuracy': 0.4099175902244956, 'roc_micro': 0.7656488926157944, 'roc_macro': 0.5534848342676337}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.8321277126465452, 'roc_macro': 0.4139440820965204}, 'mod': {'accuracy': 0.3333333333333333, 'roc_micro': 0.8321277126465452, 'roc_macro': 0.4139440820965204}, 'dlts': {'accuracy': [0.8235294117647058, 0.9411764705882353, 0.9215686274509803, 1.0, 0.9705882352941176, 0.803921568627451, 0.9803921568627451, 0.8627450980392157], 'accuracy_mean': 0.9129901960784313, 'auc': [0.4933862433862434, 0.171875, 0.6316489361702128, -1, 0.5252525252525252, 0.5609756097560976, 0.22000000000000003, 0.4301948051948052], 'auc_mean': 0.43333330282284066}}
model_done 48
done with 4 errors


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


{'predictions': [tensor([[0.1721, 0.7902, 0.0377]], device='cuda:0', grad_fn=<MeanBackward1>),
  tensor([[0.1228, 0.8337, 0.0435]], device='cuda:0', grad_fn=<MeanBackward1>),
  tensor([[0.7884, 0.0331, 0.0320, 0.0363, 0.0310, 0.0469, 0.0323]],
         device='cuda:0', grad_fn=<MeanBackward1>),
  tensor([[0.1005, 0.0600, 0.0991, 0.0534, 0.0492, 0.0832, 0.0553, 0.0913]],
         device='cuda:0', grad_fn=<MeanBackward1>)],
 '5%': [tensor([[0.0996, 0.3934, 0.0272]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.0514, 0.5729, 0.0332]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.2148, 0.0196, 0.0165, 0.0239, 0.0156, 0.0327, 0.0156]],
         device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.0531, 0.0274, 0.0592, 0.0170, 0.0175, 0.0459, 0.0148, 0.0645]],
         device='cuda:0', grad_fn=<ViewBackward0>)],
 '95%': [tensor([[0.3950, 0.8722, 0.1974]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.2223, 0.9080, 0.2043]], device='cuda:0', grad_fn=<ViewBac

In [11]:
t2_arglist = [
     {'hidden_layers': [400,400], 'dropout': 0.9, 'input_dropout': 0.1},
    {'hidden_layers': [400], 'dropout': .7, 'input_dropout': .2}
]
m2_list, m2_ensemble = train_model_ensemble(t2_arglist,2,
                                            weight_options=[[1,1,1,10],[2,2,0.1,1]],
                                            weight_baseline_options=[[1,1,1,10],[2,2,0.1,1]],
                                            n_bags=5)
emodel2 = TransitionEnsemble(m2_list,m2_ensemble)
emodel2(transition_sample(2)[0][0].view(1,-1))

/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


here


/usr/local/lib/python3.8/dist-packages/imblearn/over_sampling/_smote/base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


best loss 3.2613439559936523 {'pd': {'accuracy': 0.5, 'roc_micro': 0.9418513689700131, 'roc_macro': -1}, 'nd': {'accuracy': 0.366165611814346, 'roc_micro': 0.7496450999735329, 'roc_macro': 0.5843154804663951}, 'mod': {'accuracy': 0.366165611814346, 'roc_micro': 0.7496450999735329, 'roc_macro': 0.5843154804663951}, 'dlts': {'accuracy': [0.952054794520548, 0.9794520547945206, 0.9657534246575342, 1.0, 1.0, 0.9178082191780822, 1.0, 0.9657534246575342], 'accuracy_mean': 0.9726027397260275, 'auc': [0.7194244604316546, 0.5827505827505828, 0.5631205673758866, -1, -1, 0.6990049751243781, -1, 0.5914893617021277], 'auc_mean': 0.6311579894769259}}


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


...

best loss 3.5688531398773193 {'pd': {'accuracy': 0.5, 'roc_micro': 0.9404700920884909, 'roc_macro': -1}, 'nd': {'accuracy': 0.3314553062502059, 'roc_micro': 0.7257602321504141, 'roc_macro': 0.5309544673347794}, 'mod': {'accuracy': 0.3314553062502059, 'roc_micro': 0.7257602321504141, 'roc_macro': 0.5309544673347794}, 'dlts': {'accuracy': [0.9484536082474226, 0.9828178694158075, 0.9621993127147767, 1.0, 0.9896907216494846, 0.9347079037800687, 1.0, 0.9725085910652921], 'accuracy_mean': 0.9737972508591066, 'auc': [0.6594202898550725, 0.7342657342657343, 0.5717532467532467, -1, 0.6666666666666666, 0.6279024767801857, -1, 0.6369257950530035], 'auc_mean': 0.6494890348956516}}
model_done 47


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning

best loss 2.912658929824829 {'pd': {'accuracy': 0.5, 'roc_micro': 0.9327868852459017, 'roc_macro': -1}, 'nd': {'accuracy': 0.3481529911735861, 'roc_micro': 0.7396103640242495, 'roc_macro': 0.5145925653672233}, 'mod': {'accuracy': 0.3481529911735861, 'roc_micro': 0.7396103640242495, 'roc_macro': 0.5145925653672233}, 'dlts': {'accuracy': [0.9635761589403974, 0.9735099337748344, 0.9668874172185431, 1.0, 0.9900662251655629, 0.9370860927152318, 1.0, 0.9768211920529801], 'accuracy_mean': 0.9759933774834437, 'auc': [0.6532333645735707, 0.5935374149659864, 0.4743150684931507, -1, 0.5317725752508361, 0.633438720476102, -1, 0.6479418886198547], 'auc_mean': 0.5890398387299167}}
model_done 48
done with 3 errors


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


{'predictions': [tensor([[0.7880, 0.1807, 0.0313]], device='cuda:0', grad_fn=<MeanBackward1>),
  tensor([[0.4091, 0.5344, 0.0565]], device='cuda:0', grad_fn=<MeanBackward1>),
  tensor([[0.1839, 0.6888, 0.0728, 0.0546]], device='cuda:0',
         grad_fn=<MeanBackward1>),
  tensor([[0.0672, 0.0595, 0.0626, 0.0457, 0.0573, 0.0668, 0.0543, 0.0502]],
         device='cuda:0', grad_fn=<MeanBackward1>)],
 '5%': [tensor([[0.6642, 0.0883, 0.0126]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.3257, 0.3912, 0.0205]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.0205, 0.4145, 0.0293, 0.0137]], device='cuda:0',
         grad_fn=<ViewBackward0>),
  tensor([[0.0185, 0.0134, 0.0180, 0.0033, 0.0089, 0.0125, 0.0028, 0.0073]],
         device='cuda:0', grad_fn=<ViewBackward0>)],
 '95%': [tensor([[0.8817, 0.2677, 0.0711]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.5549, 0.6294, 0.0895]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.2922, 0.9176, 0.1924, 0.1

In [12]:
t3_arglist = [
     {'hidden_layers': [50], 'dropout': 0.5, 'input_dropout': 0},
]
m3_list, m3_ensemble = train_model_ensemble(t3_arglist,3,n_bags=5,
                                            weight_options=[[1,1,1,1],[1,2,2,1]],
                                            weight_baseline_options=[[1,1,1,1],[1,2,2,1]],
                                           )
emodel3 = EndpointEnsemble(m3_list,m3_ensemble)
emodel3(transition_sample(3)[0][0].view(1,-1))

/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


here


/usr/local/lib/python3.8/dist-packages/imblearn/over_sampling/_smote/base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


best loss 1.7472749948501587 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.12755018, 'auc': 0.4697115384615385}, 'FT': {'accuracy': -1, 'mse': 0.17344067, 'auc': 0.6688596491228069}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.129351, 'auc': 0.8048076923076923}, 'LRC': {'accuracy': -1, 'mse': 0.109962866, 'auc': 0.5577608142493639}}


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


...

best loss 1.6866246461868286 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.12341218, 'auc': 0.5728914336049329}, 'FT': {'accuracy': -1, 'mse': 0.14372012, 'auc': 0.7144444444444444}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.13668396, 'auc': 0.7038852913968547}, 'LRC': {'accuracy': -1, 'mse': 0.119112425, 'auc': 0.6398734177215191}}
model_done 23


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


best loss 3.567760944366455 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.13670115, 'auc': 0.44318181818181823}, 'FT': {'accuracy': -1, 'mse': 0.175765, 'auc': 0.5695238095238095}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.163743, 'auc': 0.655873015873016}, 'LRC': {'accuracy': -1, 'mse': 0.13793992, 'auc': 0.5982426303854875}}
model_done 24
done with 3 errors


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


{'predictions': tensor([[0.7544, 0.3500, 0.3521, 0.7760]], device='cuda:0',
        grad_fn=<MeanBackward1>),
 '5%': tensor([[0.6371, 0.2004, 0.2581, 0.7279]], device='cuda:0',
        grad_fn=<ViewBackward0>),
 '95%': tensor([[0.8350, 0.4518, 0.4555, 0.8740]], device='cuda:0',
        grad_fn=<ViewBackward0>)}

In [13]:
len(emodel3.base_models)

4

In [14]:
def gridsearch_attention_transition_models(state=1,attentions=[False]):
    hiddenlayers = [[10],[50],[100],[500],[1000],[10,10],[50,50],[100,100],[500,500],[1000,1000]]
    model_arglist = [{'hidden_layers': h} for h in hiddenlayers]
    best_loss = 100000000000
    best_metrics = {}
    best_args = {}
    best_model = None
    k = 0
    for margs in model_arglist:
        args = {k:v for k,v in margs.items()}
        for use_smote in [True,False]:
            for dropout in [0,.1,.25,.5,.9]:
                args['dropout'] = dropout
                for input_dropout in [0,.1,.25,.5]:
                    args['input_dropout'] = input_dropout
                    model,m_loss,m_metrics = train_state(
                        model_args=args,
                        state=state,
                        use_smote=use_smote,
                        use_attention=False,
                        verbose=False)
                    print('done',k,m_loss)
                    k+=1
                    if m_loss < best_loss:
                        best_loss = m_loss
                        best_metrics  = m_metrics
                        best_model = model
                        best_args = {k:v for k,v in args.items()}
                        best_args['smote'] = use_smote
                        print('_++++++++++New Best++++____')
                        print(best_loss)
                        print(best_metrics)
                        print(best_args)
                        print(use_smote)
                        print('___________')
                        print('++++++++')
                        print()
    print('_________')
    print('+++++++++++')
    print('best stuff',best_loss)
    print(best_metrics)
    print(best_args)
    return best_model
# model1 = gridsearch_attention_transition_models(1)
# model1

In [15]:
# model2 = gridsearch_attention_transition_models(2)
# model2

In [16]:
# model3 = gridsearch_attention_transition_models(3)
# model3

In [17]:
# def load_trained_models():
#     files = Const.tuned_transition_models
#     decision_file = Const.tuned_decision_model
#     [model1,model2,model3] = [torch.load(file) for file in files]
#     decision_model = torch.load(decision_file)
#     return decision_model, model1,model2,model3
# _, model1, model2, model3 =load_trained_models()

In [ ]:

def shuffle_col(v,col=None):
    if col is None:
        col = np.random.choice([i for i in range(v.shape[1])])
    idx = torch.randperm(v.shape[0])
    vv = torch.clone(v)
    vv[:,col] = vv[idx,col]
    return vv
    
def train_decision_model(
    tmodel1,
    tmodel2,
    tmodel3,
    use_default_split=True,
    use_bagging_split=False,
    lr=.0001,
    epochs=10000,
    patience=50,
    weights=[-1,1,1,-1], #realtive weight of survival, feeding tube, aspiration, andl lrc
    imitation_weights=[.5,1,1],#weights of imitation decisions, because ic overtrains too quickly
    imitation_weight=0.1,
    shufflecol_chance = 0.1,
    reward_weight=1,
    split=.7,
    resample_training=False,
    save_path='../data/models/',
    file_suffix='',
    use_attention=True,
    verbose=True,
    threshold_decisions=False,#convert decisiosn to binary in simulation, usually breaks it
    use_smote=False,
    validate_with_memory=True,
    **model_kwargs,
):
    tmodel1.eval()
    tmodel2.eval()
    tmodel3.eval()

    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)
    true_ids = train_ids + test_ids #for saving memory without upsampling
    if use_smote:
        dataset = DTDataset(use_smote=True,smote_ids = train_ids)
        train_ids = [i for i in dataset.processed_df.index.values if i not in test_ids]
    else:
        dataset = DTDataset()
    data = dataset.processed_df.copy()
    
    def get_dlt(state):
        if state == 2:
            return data[Const.dlt2].copy()
        d = data[Const.dlt1].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_pd(state):
        if state == 2:
            return data[Const.primary_disease_states2].copy()
        d = data[Const.primary_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_nd(state):
        if state == 2:
            return data[Const.nodal_disease_states2].copy()
        d = data[Const.nodal_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_cc(state):
        res = data[Const.ccs].copy()
        if state == 1:
            res.values[:,:] = np.zeros(res.values.shape)
        return res
    
    def get_mod(state):
        res = data[Const.modifications].copy()
        #this should have an ic condition but we don't use it anumore anywa
        return res
        
    outcomedf = data[Const.outcomes]
    baseline = dataset.get_state('baseline')
    
    def formatdf(d,dids=train_ids):
        d = df_to_torch(d.loc[dids]).to(model.get_device())
        return d
    
    def makegrad(v):
        if not v.requires_grad:
            v.requires_grad=True
        return v
    
    if use_attention:
        model = DecisionAttentionModel(baseline.shape[1],**model_kwargs)
    else:
        model_kwargs = {k:v for k,v in model_kwargs.items() if 'attention' not in k and 'embed' not in k}
        model = DecisionModel(baseline.shape[1],**model_kwargs)

    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    
    save_file = save_path + 'model_' + model.identifier +'_hash' + hashcode + file_suffix + '.tar'
    model.fit_normalizer(df_to_torch(baseline.loc[train_ids]))
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)

    def outcome_loss(ypred):
        l = torch.mul(torch.mean(ypred[:,0]),weights[0])
#         for i,weight in enumerate(weights[1:]):
#             newloss = torch.mul(torch.mean(ypred[:,i]),weight)
#             l = torch.add(l,newloss)
        l = torch.mul(torch.mean(-1*(ypred[:,0] - 1)),-weights[0])
        for i,weight in enumerate(weights[1:]):
            #weights with negative values will invert the outcome so e.g. Regional control becomes no regional control
            #so the penaly is correct
            if weight > 0:
                newloss = torch.mul(torch.mean(ypred[:,i]),weight)
            else:
                newloss = torch.mul(torch.mean(-1*(ypred[:,i] - 1)),-weights[i])
            l = torch.add(l,newloss)
        return l
    
    mse = torch.nn.MSELoss()
    nllloss = torch.nn.NLLLoss()
    bce = torch.nn.BCELoss()
    
    def compare_decisions(d1,d2,d3,ids):
#         ypred = np.concatenate([dd.cpu().detach().numpy().reshape(-1,1) for dd in [d1,d2,d3]],axis=1)
        ytrue = df_to_torch(outcomedf.loc[ids])
        dloss = bce(d1.view(-1),ytrue[:,0])
        dloss += bce(d2.view(-1),ytrue[:,1])
        dloss += bce(d3,view(-1),ytrue[:,2])
        return dloss
        
    def remove_decisions(df):
        cols = [c for c in df.columns if c not in Const.decisions ]
        ddf = df[cols]
        return ddf
    
    makeinput = lambda step,dids: df_to_torch(remove_decisions(dataset.get_input_state(step=step,ids=dids)))
    threshold = lambda x: torch.gt(x,torch.rand(x.shape[0])).type(torch.FloatTensor)

    
    #save the inputs from the whole dataset for future reference
    if use_attention:
        full_data = []
        for mstep in [0,1,2]:
            full_data_step = [baseline, get_dlt(min(mstep,1)),
                         get_dlt(mstep),get_pd(mstep),get_nd(mstep),get_cc(mstep),get_mod(mstep)]
            full_data_step = torch.cat([formatdf(fd,true_ids) for fd in full_data_step],axis=1)
            full_data.append(full_data_step)
        full_data = torch.stack(full_data)
        model.save_memory(full_data)
        print(full_data.shape)
    def step(train=True):
        if train:
            model.train(True)
            tmodel1.train(True)
            tmodel2.train(True)
            tmodel3.train(True)
            optimizer.zero_grad()
            ids = train_ids
        else:
            ids = test_ids
            model.eval()
            tmodel1.eval()
            tmodel2.eval()
            tmodel3.eval()
            
            
        ytrain = df_to_torch(outcomedf.loc[ids])
        #imitation losses and decision 1
        xxtrained = [baseline, get_dlt(0),get_dlt(0),get_pd(0),get_nd(0),get_cc(0),get_mod(0)]
        xxtrain = [formatdf(xx,ids) for xx in xxtrained]
        
        use_memory = (not train) and validate_with_memory
        if use_attention:
            o1 = model(torch.cat(xxtrain,axis=1),position=0,use_saved_memory = use_memory)
        else:
            o1 = model(torch.cat(xxtrain,axis=1),position=0)
        decision1_imitation = o1[:,3]
        decision1 = o1[:,0]
        if threshold_decisions:
            decision1 = threshold(decision1)

        imitation_loss1 = bce(decision1_imitation,ytrain[:,0])
        imitation_loss1 = torch.mul(imitation_loss1,imitation_weights[0])
        
        x1_imitation = [baseline, get_dlt(1),get_dlt(0),get_pd(1),get_nd(1),get_cc(1),get_mod(1)]
        x1_imitation = [formatdf(xx1,ids) for xx1 in x1_imitation]
        
        if use_attention:
            decision2_imitation = model(torch.cat(x1_imitation,axis=1),position=1,use_saved_memory = use_memory)[:,4]
        else:
            decision2_imitation = model(torch.cat(x1_imitation,axis=1),position=1)[:,4]
            
        imitation_loss2 =  bce(decision2_imitation,ytrain[:,1])
        imitation_loss2 = torch.mul(imitation_loss2,imitation_weights[1])
        
        x2_imitation = [baseline, get_dlt(1),get_dlt(2),get_pd(2),get_nd(2),get_cc(2),get_mod(2)]
        
        x2_imitation = [formatdf(xx2,ids) for xx2 in x2_imitation]
        
        if use_attention:
            decision3_imitation = model(torch.cat(x2_imitation,axis=1),position=2,use_saved_memory = use_memory)[:,5]
        else:
            decision3_imitation = model(torch.cat(x2_imitation,axis=1),position=2)[:,5]
        imitation_loss3 = bce(decision3_imitation,ytrain[:,2])
        imitation_loss3 = torch.mul(imitation_loss3,imitation_weights[2])
        
        #reward decisions
        xx1 = makeinput(1,ids)
        xx2 = makeinput(2,ids)
        xx3 = makeinput(3,ids)

        xx1 = makegrad(xx1)
        xx2 = makegrad(xx2)
        xx3  = makegrad(xx3)
        baseline_train_base = formatdf(baseline,ids)
            
        baseline_train = torch.clone(baseline_train_base)

        
        xi1 = torch.cat([xx1,decision1.view(-1,1)],axis=1)
        [ypd1, ynd1, ymod, ydlt1] = tmodel1(xi1)['predictions']
        
        d1_thresh = torch.gt(decision1.view(-1,1),.5).to(ypd1.device)
        d1_scale = torch.cat([d1_thresh,d1_thresh,torch.ones(d1_thresh.view(-1,1).shape).to(ypd1.device)],dim=1)
        ypd1= torch.mul(ypd1,d1_scale)
        ynd1= torch.mul(ynd1,d1_scale)
        
        x1 = [baseline_train,ydlt1,formatdf(get_dlt(0),ids),ypd1,ynd1,formatdf(get_cc(1),ids),ymod]
        x1=[xx1.to(model.get_device()) for xx1 in x1]
        if use_attention:
            decision2 = model(torch.cat(x1,axis=1),position=1,use_saved_memory = use_memory)[:,1] 
        else:
            decision2 = model(torch.cat(x1,axis=1),position=1)[:,1] 
            
        if threshold_decisions:
            decision2 = threshold(decision2)
            
        xi2 = torch.cat([xx2,decision1.view(-1,1),decision2.view(-1,1)],axis=1)
        [ypd2,ynd2,ycc,ydlt2] = tmodel2(xi2)['predictions']

        x2 = [baseline_train,ydlt1,ydlt2,ypd2,ynd2,ycc,ymod]
        x2 = [xx2.to(model.get_device()) for xx2 in x2]
        if use_attention:
            decision3 = model(torch.cat(x2,axis=1),position=2,use_saved_memory = use_memory)[:,2]
        else:
            decision3 = model(torch.cat(x2,axis=1),position=2)[:,2]
        if threshold_decisions:
            decision3 = threshold(decision3)
            

        xi3 = torch.cat([xx3,decision1.view(-1,1),decision2.view(-1,1),decision3.view(-1,1)],axis=1)
        outcomes = tmodel3(xi3)['predictions']

        if not train and verbose:
            print(torch.mean(outcomes,dim=0))
        reward_loss = outcome_loss(outcomes)

        loss = torch.add(imitation_loss1,imitation_loss2)
        loss = torch.add(loss,imitation_loss3)
        loss = torch.mul(loss,imitation_weight/3)
        loss = torch.add(loss,torch.mul(reward_loss,reward_weight))
        losses = [imitation_loss1+imitation_loss2+imitation_loss3,reward_loss]
        
        if not train:
            distributions = [torch.mean(torch.gt(d,.5).type(torch.FloatTensor)) for d in [decision1,decision2,decision3]]
            if verbose:
                print('decisions',distributions)

        if train:
            loss.backward()
            optimizer.step()
            return losses
        else:
            scores = []
            for i,decision in enumerate([decision1_imitation,decision2_imitation,decision3_imitation]):
                dec = decision.cpu().detach().numpy()
                dec0 = (dec > .5).astype(int)
                out = ytrain[:,i].cpu().detach().numpy()
                acc = accuracy_score(out,dec > .5)
                auc = roc_auc_score(out,dec)
                scores.append({'decision': i,'accuracy': acc,'auc': auc})
            return losses, scores, distributions
        
    best_val_loss = torch.tensor(1000000000.0)
    steps_since_improvement = 0
    best_val_score = {}
    
    for epoch in range(epochs):
        losses = step(True)
        val_losses,val_metrics,val_distributions = step(False)

        aucsum = np.sum([d['auc'] for d in val_metrics])
        vl = val_losses[1] - aucsum
        if verbose:
            print('______epoch',str(epoch),'_____')
            print('val reward',val_losses[1].item())
            print('imitation reward', val_losses[0].item())
            print('distributions',val_distributions)
            print(val_metrics)
        if vl < best_val_loss:
            best_val_loss = vl
            best_val_score = val_metrics
            best_val_distributions = val_distributions
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if steps_since_improvement > patience:
            break
    print('++++++++++Final+++++++++++')
    print('best',best_val_loss)
    print(best_val_score)
    model.load_state_dict(torch.load(save_file))
    model.eval()
    return model, best_val_score, best_val_loss, best_val_distributions

from Models import *
args = {
    'hidden_layers': [50,50], 
    'attention_heads': [2,2],
    'embed_size': 120, 
    'dropout': 0.5, 
    'input_dropout': 0.2, 
    'shufflecol_chance':  0.2,
}

#1.8424
decision_model, _, _, _ = train_decision_model(
    emodel1,emodel2,emodel3,
    lr=.0001,
    use_attention=True,
    imitation_weight=1,
    reward_weight=1,
    validate_with_memory=True,
    use_smote=False,
    threshold_decisions=True,
    **args)

/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


torch.Size([3, 536, 88])
tensor([0.7676, 0.2295, 0.2376, 0.7581], grad_fn=<MeanBackward1>)
decisions [tensor(0.4795), tensor(0.6644), tensor(0.3973)]
______epoch 0 _____
val reward 0.4670783281326294
imitation reward 1.6254088878631592
distributions [tensor(0.4795), tensor(0.6644), tensor(0.3973)]
[{'decision': 0, 'accuracy': 0.684931506849315, 'auc': 0.6413461538461538}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.45723684210526316}, {'decision': 2, 'accuracy': 0.4383561643835616, 'auc': 0.6278846153846154}]
tensor([0.7694, 0.2277, 0.2400, 0.7531], grad_fn=<MeanBackward1>)
decisions [tensor(0.4726), tensor(0.7260), tensor(0.4658)]
______epoch 1 _____
val reward 0.46767646074295044
imitation reward 1.601456880569458
distributions [tensor(0.4726), tensor(0.7260), tensor(0.4658)]
[{'decision': 0, 'accuracy': 0.6986301369863014, 'auc': 0.639423076923077}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.45614035087719296}, {'decision': 2, 'accuracy': 0.4794520547945205,

tensor([0.7685, 0.2329, 0.2388, 0.7591], grad_fn=<MeanBackward1>)
decisions [tensor(0.3767), tensor(0.7329), tensor(0.3493)]
______epoch 17 _____
val reward 0.471734881401062
imitation reward 1.336975336074829
distributions [tensor(0.3767), tensor(0.7329), tensor(0.3493)]
[{'decision': 0, 'accuracy': 0.8082191780821918, 'auc': 0.6225961538461539}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.5043859649122807}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.6673076923076924}]
tensor([0.7664, 0.2359, 0.2402, 0.7559], grad_fn=<MeanBackward1>)
decisions [tensor(0.3219), tensor(0.7329), tensor(0.3493)]
______epoch 18 _____
val reward 0.4761424660682678
imitation reward 1.3269087076187134
distributions [tensor(0.3219), tensor(0.7329), tensor(0.3493)]
[{'decision': 0, 'accuracy': 0.8287671232876712, 'auc': 0.6221153846153846}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.504111842105263}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.67115384615384

tensor([0.7694, 0.2316, 0.2373, 0.7625], grad_fn=<MeanBackward1>)
decisions [tensor(0.4452), tensor(0.7055), tensor(0.3014)]
______epoch 34 _____
val reward 0.4689490795135498
imitation reward 1.2270662784576416
distributions [tensor(0.4452), tensor(0.7055), tensor(0.3014)]
[{'decision': 0, 'accuracy': 0.8561643835616438, 'auc': 0.6423076923076924}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.5383771929824561}, {'decision': 2, 'accuracy': 0.8356164383561644, 'auc': 0.7051282051282052}]
tensor([0.7671, 0.2334, 0.2382, 0.7605], grad_fn=<MeanBackward1>)
decisions [tensor(0.3493), tensor(0.7260), tensor(0.2808)]
______epoch 35 _____
val reward 0.4716377854347229
imitation reward 1.2231805324554443
distributions [tensor(0.3493), tensor(0.7260), tensor(0.2808)]
[{'decision': 0, 'accuracy': 0.8561643835616438, 'auc': 0.6442307692307693}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.5416666666666666}, {'decision': 2, 'accuracy': 0.8356164383561644, 'auc': 0.7076923076923

tensor([0.7725, 0.2358, 0.2360, 0.7686], grad_fn=<MeanBackward1>)
decisions [tensor(0.4041), tensor(0.7603), tensor(0.2260)]
______epoch 51 _____
val reward 0.4718106985092163
imitation reward 1.1796796321868896
distributions [tensor(0.4041), tensor(0.7603), tensor(0.2260)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6745192307692308}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.5424890350877194}, {'decision': 2, 'accuracy': 0.8287671232876712, 'auc': 0.746474358974359}]
tensor([0.7711, 0.2373, 0.2374, 0.7652], grad_fn=<MeanBackward1>)
decisions [tensor(0.3493), tensor(0.7945), tensor(0.2260)]
______epoch 52 _____
val reward 0.4747864603996277
imitation reward 1.1780303716659546
distributions [tensor(0.3493), tensor(0.7945), tensor(0.2260)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6711538461538462}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.5427631578947367}, {'decision': 2, 'accuracy': 0.8287671232876712, 'auc': 0.74839743589743

...

tensor([0.7663, 0.2286, 0.2395, 0.7560], grad_fn=<MeanBackward1>)
decisions [tensor(0.4178), tensor(0.6507), tensor(0.3836)]
______epoch 255 _____
val reward 0.46802330017089844
imitation reward 1.0650317668914795
distributions [tensor(0.4178), tensor(0.6507), tensor(0.3836)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6495192307692308}, {'decision': 1, 'accuracy': 0.7671232876712328, 'auc': 0.6535087719298246}, {'decision': 2, 'accuracy': 0.8356164383561644, 'auc': 0.8522435897435898}]
tensor([0.7691, 0.2266, 0.2377, 0.7569], grad_fn=<MeanBackward1>)
decisions [tensor(0.5000), tensor(0.6712), tensor(0.4110)]
______epoch 256 _____
val reward 0.4642922878265381
imitation reward 1.0651957988739014
distributions [tensor(0.5000), tensor(0.6712), tensor(0.4110)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6480769230769231}, {'decision': 1, 'accuracy': 0.7671232876712328, 'auc': 0.6543311403508771}, {'decision': 2, 'accuracy': 0.8356164383561644, 'auc': 0.85160256

tensor([0.7686, 0.2303, 0.2387, 0.7595], grad_fn=<MeanBackward1>)
decisions [tensor(0.4315), tensor(0.7123), tensor(0.3356)]
______epoch 272 _____
val reward 0.4689926505088806
imitation reward 1.063920497894287
distributions [tensor(0.4315), tensor(0.7123), tensor(0.3356)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6427884615384616}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6600877192982456}, {'decision': 2, 'accuracy': 0.8287671232876712, 'auc': 0.8503205128205128}]
tensor([0.7720, 0.2326, 0.2381, 0.7611], grad_fn=<MeanBackward1>)
decisions [tensor(0.4658), tensor(0.7671), tensor(0.3630)]
______epoch 273 _____
val reward 0.4707452654838562
imitation reward 1.0640840530395508
distributions [tensor(0.4658), tensor(0.7671), tensor(0.3630)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6423076923076922}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6603618421052632}, {'decision': 2, 'accuracy': 0.8287671232876712, 'auc': 0.85}]
tenso

tensor([0.7683, 0.2240, 0.2378, 0.7568], grad_fn=<MeanBackward1>)
decisions [tensor(0.4863), tensor(0.6575), tensor(0.3973)]
______epoch 289 _____
val reward 0.461800217628479
imitation reward 1.066174030303955
distributions [tensor(0.4863), tensor(0.6575), tensor(0.3973)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6475961538461539}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6600877192982456}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.8516025641025642}]
tensor([0.7631, 0.2238, 0.2389, 0.7557], grad_fn=<MeanBackward1>)
decisions [tensor(0.4041), tensor(0.6164), tensor(0.3425)]
______epoch 290 _____
val reward 0.4626725912094116
imitation reward 1.066080093383789
distributions [tensor(0.4041), tensor(0.6164), tensor(0.3425)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6514423076923077}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6600877192982456}, {'decision': 2, 'accuracy': 0.8287671232876712, 'auc': 0.8516025641025

tensor([0.7672, 0.2273, 0.2361, 0.7605], grad_fn=<MeanBackward1>)
decisions [tensor(0.4521), tensor(0.6438), tensor(0.3219)]
______epoch 306 _____
val reward 0.4633473753929138
imitation reward 1.0711594820022583
distributions [tensor(0.4521), tensor(0.6438), tensor(0.3219)]
[{'decision': 0, 'accuracy': 0.8972602739726028, 'auc': 0.6423076923076922}, {'decision': 1, 'accuracy': 0.7465753424657534, 'auc': 0.6641995614035088}, {'decision': 2, 'accuracy': 0.8356164383561644, 'auc': 0.8509615384615385}]
tensor([0.7675, 0.2259, 0.2405, 0.7561], grad_fn=<MeanBackward1>)
decisions [tensor(0.4384), tensor(0.6575), tensor(0.3630)]
______epoch 307 _____
val reward 0.46642786264419556
imitation reward 1.0715417861938477
distributions [tensor(0.4384), tensor(0.6575), tensor(0.3630)]
[{'decision': 0, 'accuracy': 0.8972602739726028, 'auc': 0.6423076923076922}, {'decision': 1, 'accuracy': 0.7465753424657534, 'auc': 0.6644736842105263}, {'decision': 2, 'accuracy': 0.8356164383561644, 'auc': 0.85128205

tensor([0.7669, 0.2266, 0.2381, 0.7584], grad_fn=<MeanBackward1>)
decisions [tensor(0.4247), tensor(0.6644), tensor(0.3699)]
______epoch 323 _____
val reward 0.46467602252960205
imitation reward 1.068544626235962
distributions [tensor(0.4247), tensor(0.6644), tensor(0.3699)]
[{'decision': 0, 'accuracy': 0.8972602739726028, 'auc': 0.6485576923076923}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6721491228070176}, {'decision': 2, 'accuracy': 0.8424657534246576, 'auc': 0.8535256410256411}]
tensor([0.7665, 0.2252, 0.2389, 0.7536], grad_fn=<MeanBackward1>)
decisions [tensor(0.4863), tensor(0.6644), tensor(0.4315)]
______epoch 324 _____
val reward 0.46404069662094116
imitation reward 1.0683696269989014
distributions [tensor(0.4863), tensor(0.6644), tensor(0.4315)]
[{'decision': 0, 'accuracy': 0.8972602739726028, 'auc': 0.6485576923076923}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6726973684210527}, {'decision': 2, 'accuracy': 0.8424657534246576, 'auc': 0.85384615

In [47]:
torch.rand(1).item()

0.6044421195983887

In [19]:
def gridsearch_decision_model(m1,m2,m3,weights=[1,1,0,0]):
    model_arglist = [

        {
            'hidden_layers': [100,100],
            'attention_heads': [1,1],
        },
        {
            'hidden_layers': [50,50],
            'attention_heads': [1,1],
        },
        {
            'hidden_layers': [100,100],
            'attention_heads': [2,2],
        },
        {
            'hidden_layers': [300],
            'attention_heads': [3],
        },
        {
            'hidden_layers': [600],
            'attention_heads': [3],
        },
        {
            'hidden_layers': [300,300],
            'attention_heads': [3,3],
        },
        {
            'hidden_layers': [600,600],
            'attention_heads': [3,3],
        },
        {
            'hidden_layers': [1000,100],
            'attention_heads': [1],
        },
        {
            'hidden_layers': [1000,100],
            'attention_heads': [5],
        },
    ]
    best_loss = 100000000000
    best_metrics = {}
    best_args = {}
    best_model = None
    best_record = {}
    k = 0
    records = []
    for margs in model_arglist:
        args = {k:v for k,v in margs.items()}
        for embed_size in [0,120]:
            #embed_size = 0 skips the firt layer that makes the sizes right
            if embed_size == 0 and args['attention_heads'][0] != 1:
                continue
            args['embed_size'] = embed_size
            for dropout in [.5,.75]:
                args['dropout'] = dropout
                for input_dropout in [0,.5]:
                    args['input_dropout'] = input_dropout
                    for shufflecol_chance in [.5]:
                        for use_smote in [True,False]:
                            try:
                                args['shufflecol_chance'] = shufflecol_chance
                                model,m_metrics,m_loss,m_distribution = train_decision_model(m1,m2,m3,
                                                                              use_attention=True,
                                                                              validate_with_memory = True,
                                                                            weights=weights,
                                                                            use_smote=use_smote,
                                                                              verbose=False,
                                                                                             **args)
                                entry = {
                                    'metrics': m_metrics,
                                    'loss': m_loss,
                                    'args': args,
                                    'distribution': m_distribution,
                                    'smote': use_smote
                                }
                                records.append(entry)
                                print('done',k,m_loss)
                                print('curr best',best_loss)
                                k+=1
                                if m_loss < best_loss:
                                    best_loss = m_loss
                                    best_metrics  = m_metrics
                                    best_model = model
                                    best_args = args
                                    best_record = entry
                                    print('_++++++++++New Best++++____')
                                    print(best_loss)
                                    print(best_metrics)
                                    print(best_args)
                                    print('___________')
                                    print('++++++++')
                                    print()
                            except Exception as e:
                                print('error',e,args)
    print('_________')
    print('+++++++++++')
    print('best stuff',best_loss)
    print(best_metrics)
    print(best_args)
    return best_model, records, best_record

# from Models import *
# decision_model,records, bestr = gridsearch_decision_model(model1,model2,model3_smote,weights=[1,1,.1,0])
# records

In [20]:
decision_model

DecisionAttentionModel(
  (input_dropout): Dropout(p=0.1, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=120, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
  )
  (batchnorm): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (relu): Softplus(beta=1, threshold=20)
  (sigmoid): Sigmoid()
  (softmax): LogSoftmax(dim=1)
  (final_layer): Linear(in_features=50, out_features=6, bias=True)
  (resize_layer): Linear(in_features=92, out_features=120, bias=True)
  (attentions): ModuleList(
    (0): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=120, out_features=120, bias=True)
    )
    (1): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=50, out_features=50, bias=True)
    )
  )
  (norms): ModuleList(
    (0): LayerNorm((120,), eps=1e-05, elementwise_affine=True)
    (1): LayerNorm((50,), eps=

In [21]:
torch.save(decision_model,'../data/models/final_decision_model_' + decision_model.identifier + '.pt')
print('../data/models/final_decision_model_' + decision_model.identifier + '.pt')

../data/models/final_decision_model_statedecisions_input125_dims50,50_dropout0.1,0.5.pt


In [22]:
# torch.save(model1,'../data/models/final_transition1_model_' + emodel1.identifier + '.pt')
# torch.save(model2,'../data/models/final_transition2_model_' + emodel2.identifier + '.pt')
# torch.save(model3,'../data/models/final_outcome_model_' + emodel3.identifier + '.pt')
# print('../data/models/final_transition1_model_' + emodel1.identifier + '.pt')
# print('../data/models/final_transition2_model_' + emodel2.identifier + '.pt')
# print('../data/models/final_outcome_model_' + emodel3.identifier + '.pt')

In [33]:
decision_model.set_device('cpu')
emodel1.set_device('cpu')
emodel2.set_device('cpu')
emodel3.set_device('cpu')
torch.save(decision_model,'../resources/decision_model.pt')
torch.save(emodel1,'../resources/transition1_model.pt')
torch.save(emodel2,'../resources/transition2_model.pt')
torch.save(emodel3,'../resources/outcome_model.pt')

In [24]:
def get_default_predictions(dm):
    res  = []
    for state in [0,1,2]:
        mem = dm.memory[state]
        mem = torch.median(mem,dim=0)[0].type(torch.FloatTensor)
        val = dm(mem.reshape(1,-1),position=state)
        res.append(val.cpu().detach().numpy())
    return np.stack(res)
get_default_predictions(decision_model)

array([[[0.6159429 , 0.112011  , 0.09524623, 0.85989046, 0.05497414,
         0.17375405]],

       [[0.60916203, 0.10708054, 0.11144593, 0.8756373 , 0.04198916,
         0.216172  ]],

       [[0.50641745, 0.17178199, 0.28917038, 0.7273218 , 0.20743132,
         0.14821538]]], dtype=float32)